In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# This file contains all the main external libs we'll use

import sys
import fastai
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

sz=224
bs=64

In [ ]:
PATH='/Users/bharatkunwar/Datasets/dogs-cats/'
SAMPLE='train/cats'

In [ ]:
files = !ls {PATH}/{SAMPLE} | head

In [ ]:
files

In [ ]:
img = plt.imread(f'{PATH}/{SAMPLE}/{files[0]}')
plt.imshow(img)
img.shape

# 1— IMAGE RECOGNITION

In [ ]:
%%time
tfms = tfms_from_model(resnet34, sz)
data = ImageClassifierData.from_paths(PATH, bs=bs, tfms=tfms)
learn = ConvLearner.pretrained(resnet34, data, precompute=True)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()
plt.axvline(x=0.01, color="red");

In [ ]:
%%time
learn.fit(0.01, 3)

In [ ]:
learn.save('part1')

# 2—CONVOLUTIONAL NEURAL NETWORKS


In [ ]:
tfms = tfms_from_model(resnet34, sz, 
                       aug_tfms=transforms_side_on,
                       max_zoom=1.0)

data = ImageClassifierData.from_paths(PATH, bs=bs, tfms=tfms, num_workers=1)
learn = ConvLearner.pretrained(resnet34, data, precompute=False)
learn.load('part1')

In [ ]:
def get_augs():
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

ims = np.stack([get_augs() for i in range(12)])
plots(ims, rows=3)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()
plt.axvline(x=0.01, color="red");

In [ ]:
%%time
learn.fit(0.01, 3, cycle_len=1)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_lastlayer')

In [ ]:
learn.unfreeze()

In [ ]:
lr=np.array([0.0001,0.001,0.01])

In [ ]:
learn.fit(lr, 1, cycle_len=1, cycle_mult=2)